# 准备工作
1. 安装python
2. 通过pip install openpyxl（没安装的安装）
3. 通过pip install docxtl(没有安装的安装)
# 使用
1. 要xlsx文件，并且进行整理
2. 制作模板
    * 在不涉及列表循环的地方，在想要替换的地方通过*{{ 参会人名 }}*[其中的‘参会人名’是xlsx文件中的第一列的对应名称]来替换；
    * 在设计列表循环的地方，只留下一列，之后上下分别增加一列，并且合并单元格。然后上面的那行输入*{%tr for item in items%}*。在下面的一行输入*{%tr endfor %}*。并且在循环里面通过`{{item.参会人名}}`来进行替换。
    * **特别注意：**制作完模板要关闭掉word。
3. 配置下面这个块的内容
4. 挨个执行

In [ ]:
# 这个小块是一些配置
xlsxPath = '/Users/apple/Desktop/任务书数据.xlsx'       # 这里配置xlsx的文件路径
tplPath = '/Users/apple/Desktop/活动安排-任务书.docx'           # 模板文件的路径
namedVlue = '参会人名'                 # 在xlsx中代表姓名的栏目的名称
outPutdir = '/Users/apple/Desktop/docx/'             # 输出的文件夹名
isSimple = True             # 是否是简单的，就是一条生成一个。不是这里改False

In [ ]:
# 读取数据
from openpyxl import load_workbook
from collections import namedtuple


workbook = load_workbook(xlsxPath)
#booksheet = workbook.active                #获取当前活跃的sheet,默认是第一个sheet
sheets = workbook.get_sheet_names()         #从名称获取sheet
booksheet = workbook.get_sheet_by_name(sheets[0])
 
rows = booksheet.rows
titleList = None
contentList = []
for row in rows:
    if not titleList:
        # 通过第一行的标题生成Record对象
        line = [col.value for col in row if col.value]
        titleList = line
        # 通过namedtuple声称对象
        # print(titleList)
        Record = namedtuple('Record', titleList)
    else:
        # 先进行检查是否到了终止的地方
        if row[0].value and row[1].value:
            # 有数据
            singleList = [col.value for col in row[:len(titleList)]]
            record = Record._make(singleList)
            # print(record)
            contentList.append(record)
        else:
            break;

In [ ]:
# 模板生成
from docxtpl import DocxTemplate
import os
if not outPutdir.endswith(os.path.sep):
    outPutdir = outPutdir + os.path.sep
if not os.path.exists(outPutdir):
    os.makedirs(outPutdir)
tpl = DocxTemplate(tplPath)          # 这个是模板的地址
# 获取模板文件的后缀
fileType = tplPath[tplPath.rindex('.') + 1]
# 判断是否是多条一个文件的。
if isSimple:
    for record in contentList:
        context = record._asdict()
        tpl.render(context)
        tpl.save(outPutdir + namedVlue + fileType)
else:
    from collections import defaultdict
    complexDict = defaultdict(list)
    for record in contentList:
        complexDict[getattr(record, namedVlue)].append(record)
    for recordList in complexDict.values():
        context = recordList[0]._asdict()
        context['items'] = recordList
        print(context)
        tpl.render(context)
        tpl.save(outPutdir + namedVlue + fileType)